<a href="https://colab.research.google.com/github/waswapeter/CEE_Class-Codes-Data/blob/main/aviation_planning/airlineplanning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright © 2023 Gurobi Optimization, LLC

# Airline planning after flight disruption

Weather events are a major threat to the airline industry. The unpredictable nature of snowstorms, heavy rains, and icy runways make it difficult for aviation planners to make accurate schedules.
These events can lead to flight delays and cancellations, causing not only inconvenience for passengers but also significant financial losses for airlines.
The 2014 winter polar vortex, for instance, is estimated to have cost the industry and passengers a staggering $1.4 billion in losses due to disrupted flights [(CNBC, 2014)](https://www.cnbc.com/2014/01/08/weather-flight-disruptions-cost-14-billion-data.html).
Thus, managing weather-related issues and having contingency plans in place is critical for the success of any airline business.


<!--  <img height="227.4774774774775px" src="https://media3.giphy.com/media/gkAEM5sXCFqB465YWg/giphy.gif?cid=de9bf95e87544fzo1lm01e68i3mxxou95x5zgiewz66hwi45&amp;rid=giphy.gif&amp;ct=g" width="700px" itemtype="http://schema.skype.com/Giphy" key="gif_0"> -->

 |<img src="https://raw.githubusercontent.com/Gurobi/modeling-examples/master/aviation_planning/image_snowstorm.jpeg" width="500" align="center">|
|:--:|
| <b>Flight cancellations due to weather delays are more common than we think. Image Credits: [Travel Refund](https://travelrefund.com/articles/when-do-flights-get-cancelled-due-to-weather/) </b>|

Suppose that you are a flight planner for an airline. For a given day, you have sold tickets for flights across the country, and you have a plan for operating your aircraft fleet to service all of these flights.

Let us say that on this day, there is a weather event (such as a snowstorm) that inhibits the airports from operating at full capacity. This means that some flights have to get cancelled. When a flight will be cancelled, the aircrafts assigned to these flights have to be re-routed. So the question becomes: how can airlines decide which flights to operate/cancel and how to best re-route the aircrafts?

There is no straight-forward answer to this question, but mathematical optimization can help.


This notebook walks through the optimization problem of deciding which flights to operate and which flights to cancel after a weather disruption.
We do this by constructing a **mathematical optimization** model that reduces the revenue lost from the cancelled flights.
In this example, we are using real data in France compiled by [Amadeus](https://amadeus.com/en).

There are three parts to this notebook.
- First, we read the datasets.
- Second, we build the optimization model by defining the **decision variables**, **objective function**, and the **constraints**.
- Third, assuming a certain level of weather disruption, we solve the optimization model to find the new optimal flight plan as well as routes for the aircrafts.

## The data


**Dataset**: We use real data compiled by [Amadeus](https://amadeus.com/en), made available as part of the [ROADEF 2009 Challenge: Disruption Management for Commercial Aviation](https://www.roadef.org/challenge/2009/en/). This dataset is based on flight plans for an airline in France. For this notebook, we have pre-processed this dataset and stored the information in three parts:

- **Current flight plan:** The currently planned set of flights and their aircraft assignments assuming that there is no weather disruption (i.e., all airports operate at full capacity).
- **Aircraft starting and ending positions:** Where should each aircraft start the day and end the day? This information is necessary to ensure that aircrafts are where they need to be for the next day so that the disruption does not extend into the next day.
- **Passenger itinerary:** The number of passengers and the price per ticket sold for each flight. This information is useful to assess the revenue brought in by each flight.

Note that even though the data used in this example is from 2006, the optimization model is ambivalent of the data.
For any new flight plan and predicted future disruption levels, the model will optimally solve the routing and flight service decisions.


### Packages
First, install and import the Python packages needed for processing the data.

In [1]:
%pip install networkx matplotlib seaborn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import random
import csv
import plotly.express as px

### Flight plan without disruption

Next, read the  planned schedule of flights on the day of disruption, i.e., 7th January, 2006. Additionally, we infer the origin and destination airports for each flight, as well as the start time (when the flight departs the origin airport) and the end time (when the flight arrives at the destination airport).
We store it in a Pandas dataframe.



In [2]:
df_current_plan = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/aviation_planning/data/flight_rotations_2006-07-01.csv')
# if you run this notebook locally, you can also use
#df_current_plan = pd.read_csv('data/flight_rotations_2006-07-01.csv')

df_current_plan['start_time'] = pd.to_datetime(df_current_plan['start_time'], format='%H:%M')
df_current_plan['start_time'] = df_current_plan['start_time'].dt.time
df_current_plan['end_time'] = pd.to_datetime(df_current_plan['end_time'], format='%H:%M')
df_current_plan['end_time'] = df_current_plan['end_time'].dt.time
df_current_plan['duration'] = pd.to_datetime(df_current_plan['duration'], format='%H:%M')
df_current_plan['duration'] = df_current_plan['duration'].dt.time
df_current_plan

,flight,date,aircraft,ori,des,start_time,end_time,duration
0,1,7/1/06,TranspCom#1,CDG,ORY,00:00:00,00:30:00,00:30:00
1,73,7/1/06,TranspCom#3,ORY,CDG,00:00:00,00:30:00,00:30:00
2,2,7/1/06,TranspCom#2,CDG,ORY,00:20:00,00:50:00,00:30:00
3,74,7/1/06,TranspCom#4,ORY,CDG,00:20:00,00:50:00,00:30:00
4,75,7/1/06,TranspCom#1,ORY,CDG,00:40:00,01:10:00,00:30:00
...,...,...,...,...,...,...,...,...
603,142,7/1/06,TranspCom#4,ORY,CDG,23:00:00,23:30:00,00:30:00
604,143,7/1/06,TranspCom#1,ORY,CDG,23:20:00,23:50:00,00:30:00
605,71,7/1/06,TranspCom#3,CDG,ORY,23:20:00,23:50:00,00:30:00
606,144,7/1/06,TranspCom#2,ORY,CDG,23:40:00,00:10:00,00:30:00


How many **flights**, **airports** and **aircrafts** are in this dataset?

In [3]:
flights = df_current_plan['flight'].unique()
aircrafts = df_current_plan['aircraft'].unique()
airports = set(df_current_plan['ori'].unique()+df_current_plan['des'].unique())

print(len(flights),"flights between",len(airports),"airports operated with",len(aircrafts),"aircrafts")

608 flights between 35 airports operated with 85 aircrafts


### Visualizing the flight network

Next, we assemble the origin-destination airports for all the flights, and visualize the entire network of planned flights. We use networkx to store the inter-airport information as a *directed graph* data structure to enable this visualization.


Moreover, to reduce the size of the inputs, we can select just the top few airports (in terms of how many flights flow through them) for the rest of the notebook. The n_airports parameter selects the number of top airports to pre-select, with default set to 20 airports.


The visual below is intended to illustrate the complexity of the flight map; do not feel like you need to spend too much time analyzing it. We will drill down into specific flight paths later.

In [6]:
from IPython.display import Image, display
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from networkx.drawing.nx_agraph import to_agraph

arcs = list(df_current_plan[['ori','des']].itertuples(index=False, name=None)) # store the origin-destination pairs of all the flights

n_airports = 4 # specify how many airports to pick

G = nx.MultiDiGraph() # create an empty directed graph
G.add_edges_from(arcs) # add the origin-destination pairs to the graph as directed edges

top_airports = [i for (i,j) in sorted(G.degree, key=lambda x: x[1], reverse=True)[:n_airports]] # pre-select top airports by their degree

G = G.subgraph(top_airports) # reduce the graph to just the top few airports

# reduce the current plan dataframe to just the top few airports
df_current_plan = df_current_plan[df_current_plan['ori'].isin(top_airports)]
df_current_plan = df_current_plan[df_current_plan['des'].isin(top_airports)]

# # visualize the network
# A_graph = to_agraph(G)
# A_graph.layout('dot')
# display(A_graph)


flights = df_current_plan['flight'].unique()
aircrafts = df_current_plan['aircraft'].unique()
airports = set(df_current_plan['ori'].unique()+df_current_plan['des'].unique())

print("The reduced data has",len(flights),"flights between",len(airports),"airports operated with",len(aircrafts),"aircrafts")


The reduced data has 222 flights between 4 airports operated with 21 aircrafts


For each flight, we will store its origin airport, destination airport, starting time and ending time in dictionaries.


In [7]:

flight_origin = df_current_plan.set_index('flight')['ori'].to_dict()
flight_dest = df_current_plan.set_index('flight')['des'].to_dict()
flight_start_time = df_current_plan.set_index('flight')['start_time'].to_dict()
flight_end_time = df_current_plan.set_index('flight')['end_time'].to_dict()


### Where should aircrafts start the day and end the day?

At the start of the disruption day, each aircraft starts from a particular airport, and must end the day at a particular airport. This is to ensure that the aircraft fleet is ready and available for uninterrupted flight operations for the next day.

We now read the information on the aircraft fleet's starting position (called the **source**) and ending position (called the **sink**).

In [8]:
df_starting_positions = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/aviation_planning/data/starting_positions.csv')
# if you run this notebook locally, you can also use
#df_starting_positions = pd.read_csv('data/starting_positions.csv')
aircrafts_startpositions_airc = df_starting_positions.set_index('aircraft')['airport'].to_dict()

df_ending_positions = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/aviation_planning/data/ending_positions.csv')
# if you run this notebook locally, you can also use
#df_ending_positions = pd.read_csv('data/ending_positions.csv')
aircrafts_endpositions_airc = df_ending_positions.set_index('aircraft')['airport'].to_dict()


### Passenger itineraries

Next, we read the passenger itinerary data. For each flight, we know how many passengers booked the tickets and the cost of each seat. We will store this information in dictionaries so that we can later assess the cost of cancelling a flight.

In [9]:
df_iterinaries = pd.read_csv('https://raw.githubusercontent.com/Gurobi/modeling-examples/master/aviation_planning/data/flight_iterinaries.csv')
#df_iterinaries = pd.read_csv('data/flight_iterinaries.csv')
df_iterinaries['total_cost'] = df_iterinaries['cost']*df_iterinaries['n_pass']
flight_revenue = df_iterinaries.groupby(['flight'])['total_cost'].agg('sum').to_dict()
flight_n_pass = df_iterinaries.groupby(['flight'])['n_pass'].agg('sum').to_dict()
df_iterinaries


,cost,n_pass,flight,total_cost
0,137.5,24.0,4296.0,3300.0
1,137.5,33.0,4296.0,4537.5
2,137.5,24.0,4296.0,3300.0
3,137.5,44.0,4296.0,6050.0
4,137.5,35.0,4295.0,4812.5
...,...,...,...,...
1925,200.0,15.0,2620.0,3000.0
1926,250.0,6.0,2609.0,1500.0
1927,250.0,4.0,2609.0,1000.0
1928,325.0,16.0,5125.0,5200.0


### Create flight-to-flight transitions for each aircraft

Finally, using the current flight plan, we assess all the feasible flight-to-flight transitions. Such a transition is  essentially: after each flight, what is the next possible flight?

For two flights $f_1$ and $f_2$, the flight transition $f_1$-$f_2$ is *feasible* if the arrival time of $f_1$ is before the departure time of $f_2$, and the destination of $f_1$ is the same as the origin of $f_2$.

Based on these feasible transitions, each aircraft's route is a sequence of flight-to-flight transitions starting from the source airport to its sink airport.


For example, from the data, the aircraft A380#1 (which is an Airbus 380) starts the day at airport CFE. From here, it can either take flight 4296 (CFE-ORY) at 5:40am or flight 4298 (CFE-ORY) at 10:48am. Once it gets to ORY, it will have multiple options for the rest of the day's route. In total,  A380#1 has eight flight paths (see visualization below).

To compactly store all the feasible flight-to-flight transitions, we create a **directed acyclic graph** (DAG). The vertices are the flights, and the directed edges are feasible transitions. Use the interactive tool below to visualize the DAG for each aircraft.

In [10]:

from ipywidgets import interact, interactive, fixed, interact_manual

aircraft_flights = df_current_plan.groupby(['aircraft']).apply(lambda x: x['flight'].tolist()).to_dict()
flight_arcs_for_each_aircraft = {}
deltaplus_flightarcs = {}
deltaminus_flightarcs = {}
for a in aircraft_flights:
    aircraft_flights[a] += ['source_%s'%a,'sink_%s'%a]
    flight_origin['source_%s'%a] = aircrafts_endpositions_airc[a]
    flight_dest['source_%s'%a] = aircrafts_startpositions_airc[a]
    flight_origin['sink_%s'%a] = aircrafts_endpositions_airc[a]
    flight_dest['sink_%s'%a] = aircrafts_startpositions_airc[a]

    flight_start_time['source_%s'%a] = datetime.strptime('0:0', '%H:%M').time()
    flight_end_time['source_%s'%a] = datetime.strptime('0:0', '%H:%M').time()

    flight_start_time['sink_%s'%a] = datetime.strptime('23:59', '%H:%M').time()
    flight_end_time['sink_%s'%a] = datetime.strptime('23:59', '%H:%M').time()

    flight_arcs_for_each_aircraft[a] = []
    deltaplus_flightarcs[a] = {f: [] for f in aircraft_flights[a]}
    deltaminus_flightarcs[a] = {f: [] for f in aircraft_flights[a]}

    for f1 in aircraft_flights[a]:
        for f2 in aircraft_flights[a]:
            if f1!=f2 and flight_end_time[f1] < flight_start_time[f2] and flight_dest[f1] == flight_origin[f2]:
                flight_arcs_for_each_aircraft[a].append((f1,f2))
                deltaplus_flightarcs[a][f1].append(f2)
                deltaminus_flightarcs[a][f2].append(f1)
            # allow to connect source and target directly for the case that aircraft is not used at all
            elif str(f1).startswith('source') and str(f2).startswith('sink'):
                flight_arcs_for_each_aircraft[a].append((f1,f2))
                deltaplus_flightarcs[a][f1].append(f2)
                deltaminus_flightarcs[a][f2].append(f1)

In [11]:
# packages needed to visualize the DAG
# uncomment the whole cell in case of troubles
!apt install libgraphviz-dev
!pip install pygraphviz

def visualize_aircraft_network(x):
    G = nx.DiGraph()
    G.add_edges_from(flight_arcs_for_each_aircraft[x])
    plt.figure(figsize=(20,14))
    A_graph = to_agraph(G)
    A_graph.layout('dot')
    display(A_graph)
    plt.show()

interact(visualize_aircraft_network, x=aircraft_flights.keys())

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libgvc6-plugins-gtk
  librsvg2-common libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk librsvg2-common libxdot4
0 upgraded, 9 newly installed, 0 to remove and 49 not upgraded.
Need to get 2,434 kB of archives.
After this operation, 7,681 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgtk2.0-common all 2.24.33-2ubuntu2.1 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgtk2.0-0 amd64 2.24.33-2ubuntu2.1 [2,038 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgail18 amd64 2.24.33-2ubuntu2.1 [15.9 kB]
Get:4 http://archive.ubun

interactive(children=(Dropdown(description='x', options=('A318#4', 'A318#5', 'A318#7', 'A318#8', 'A319#11', 'A…

<function __main__.visualize_aircraft_network(x)>

## Optimization model


A weather disruption diminishes the overall capacity of the airports, measured by the number of flights that can take-off and land.
Given this reduced airport capacity, which flights should be operated, and what route should the aircrafts take?
Our goal is to create an optimal flight plan that minimizes the overall revenue loss incurred from the cancelled flights.

This decision problem is modeled using a mathematical optimization model, which finds the **best solution** according to an **objective function** such that the solution satisfies a set of **constraints**.
Here, a solution is expressed as a vector of real values or integer values called **decision variables**.
Constraints are a set of equations or inequalities written as a function of the decision variables.

In this airline business model, the objective is to minimize the overall loss from all the cancelled flights.
The decision variables decide which flights to operate/cancel, as well as construct a route for each aircraft that starts from its starting airport and ends at the airport it needs to be at the end of the day.
There are three types of constraints: (i) construct the flight route, (ii) ensure that a flight is operated only if it is in the flight route, and (iii) ensure that the number of take-offs and landings are within the diminished capacity of the airports.

### Assumptions

There are many modeling assumptions made in this notebook, as this model serves as a starting point. At the end of the notebook, we suggest potential extensions.
The following are some key assumptions.
- All airports have the same level of disruption, which is across the entire day.
- We assume that we know ahead of time the level of disruption at all airports.
- We ignore crew scheduling and maintainance issues; though this model can be extended for larger inputs with a commercial Gurobi licence.
- We do not consider how other airlines may react to the disruption.



### Input Parameters

Let us now define the input parameters and notations used for creating the model. The subscript $a$ will be used to denote each aircraft, $f$ for each flight, and $i$ for each airport.


- $N$: set of all airports
- $A$: set of all aircrafts
- $F$: set of all flights
- $F_a$: set of flights operated by aircraft $a$ in the current plan
- $E_a$: set of feasible flight-to-flight transitions for aircraft $a$
- $r_f$: revenue ($\$$) from operating flight $f$
- $(o_f,d_f)$: origin, destination airports for flight $f$
- $(C^{arr}_i,C^{dep}_i)$: maximum number of arrivals and departures in airport $i$
- $\alpha$: level of disruption

The following code loads the Gurobi python package and initiates the optimization model.
The value of $\alpha$ is set to $50\%$.


In [12]:
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
model = gp.Model("airline_disruption")

N = G.nodes()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 52.3 MB/s eta 0:00:00
Restricted license - for non-production use only - expires 2026-11-23


### Decision Variables

We now define the decision variables.
In our model, we want to do two things: pick flights to be operated by each aircraft and construct a route for each aircraft.
The following notation is used to model these decision variables.


$x_{a,f}$: $1$, if aircraft $a$ operates flight $f$; $0$, otherwise

$y_{a,f_1,f_2}$: $1$, if aircraft $a$ operates flight $f_2$ immediately after flight $f_1$; $0$, otherwise

We will now add the variables to the Gurobi model using the addVar function.

In [13]:
x, y = {}, {}
for a in aircrafts:
    for f in aircraft_flights[a]:
        x[a,f] = model.addVar(name="x_%s,%s"%(a,f), vtype=GRB.BINARY)

    for (f1,f2) in flight_arcs_for_each_aircraft[a]:
        y[a,f1,f2] = model.addVar(name="y_%s,%s,%s"%(a,f1,f2), vtype=GRB.BINARY)

model.update()


### Set the Objective: minimize the revenue from cancelled flights

<!-- Next, we will define the objective function: we want to maximizing the **net revenue**. The revenue from sales in each region is calculated by the price of an avocado in that region multiplied by the quantity sold there. There are two types of costs incurred: the wastage costs for excess unsold avocados and the cost of transporting the avocados to the different regions.

The net revenue is the sales revenue subtracted by the total costs incurred. We assume that the purchase costs are fixed and are not incorporated in this model. -->

Our goal is to **minimize** the total **lost revenue** from the cancelled flights.
We capture this objective as a function of the decision variables.
Note that a flight is cancelled if $x_{a,f}$ is set to $0$.
The revenue lost from the flight is given by $(1-x_{a,f}) * r_f$.
Hence, the overall lost revenue across all aircrafts and cancelled flights is given by,


<!-- \begin{aligned}
\textrm{Maximize } \ \sum_{f \in flights} \ \sum_{a \in aircrafts} \ x_{a,f} * r_f
\end{aligned} -->

\begin{aligned}
\textrm{Minimize } \ \sum_{a \in aircrafts} \ \sum_{f \in F_a} \ (1-x_{a,f}) * r_f
\end{aligned}

We now add this objective function to the model using the setObjective function.



In [14]:
objective = gp.quicksum((1-x[a,f])*flight_revenue[f] for a in aircrafts for f in aircraft_flights[a] if f in flight_revenue) # operating cost
model.setObjective(objective, sense=GRB.MINIMIZE)

### Constraint #1: construct each aircraft's flight path


An aircraft begins the day from its starting airport (source) and ends the day at its final airport (sink). Its route during the day is constructed using the **y** decision variables.

We do this by considering three cases for each aircraft: its starting flight, an intermediary flight, and its ending flight.

When a flight leaves its starting airport, we ensure that it can leave exactly once.
The flights in the set $\delta^+(source_a)$ gives the set of all candidate "first-flights" for the aircraft.
We ensure that exactly one of these flights is taken using the following equality for each aircraft $a$.

\begin{aligned}
\sum_{f' \in \delta^+(source_a)} y_{a,source_a,f'} &= 1
\end{aligned}

Similarly, a flight arrives at its final airport, we ensure that it enters the airport exactly once.
The flights in the set $\delta^-(sink_a)$ gives the set of all candidate "last-flights" for the aircraft, and we ensure that exactly one of these flights is taken.

\begin{aligned}
\sum_{f' \in \delta^-(sink_a)} y_{a,f',sink_a} &= 1
\end{aligned}

For every intermediary flight $f$ in $F_a$ (that is neither the starting nor the ending flight), we ensure that the number of preceding and succeeding flights are the same. This is necessary to ensure the continuity of the flight path.
The following constraints are for each aircraft $a$ and  intermediary flight $f$ in $F_a$.

\begin{aligned}
\sum_{f' \in \delta^+(f)} y_{a,f,f'} &= \sum_{f' \in \delta^-(i)} y_{a,f',f}
\end{aligned}

In optimization modeling, these types of constraints are called **flow-balance** constraints.
These are used to model many famous problems such as shortest path, maximum flow problem, and the traveling salesman problem. Read more [here](https://web.mit.edu/15.053/www/AMP-Chapter-08.pdf).
The following code adds these constraints to the model one at a time.

In [15]:
for a in aircrafts:
    model.addConstr(sum(y[a,'source_%s'%a,f2] for f2 in deltaplus_flightarcs[a]['source_%s'%a]) == 1)
    model.addConstr(sum(y[a,f1,'sink_%s'%a] for f1 in deltaminus_flightarcs[a]['sink_%s'%a]) == 1)
    for f in aircraft_flights[a]:
        if str(f)[0] != 's':
            model.addConstr(sum(y[a,f,f2] for f2 in deltaplus_flightarcs[a][f]) == sum(y[a,f1,f] for f1 in deltaminus_flightarcs[a][f]))

### Constraint #2: a flight is operated only if it is traversed by an aircraft

Next, we make sure that a flight $f$ is operated by an aircraft $a$ only if $f$ is in the route taken by $a$.
The quantity $\sum_{f' \in \delta^+(f)} y_{a,f,f'}$ gives us the number of arcs that leave flight $f$; there can be either $0$ arcs or $1$ arc.
If this quantity is $0$, then aircraft $f$ does not traverse flight $f$, and we set $x_{a,f}$ to be $0$.
This constraint can be mathematically expressed by the following inequality for each aircraft $a$ and flight $f$ in $F_a$

\begin{aligned}  
x_{a,f} &\leq \sum_{f'\ \textrm{in }\ \delta^+(f)} y_{a,f,f'}
\end{aligned}

Let us now add these constraints to the model.


In [16]:
for a in aircrafts:
    for f in aircraft_flights[a]:
        model.addConstr(x[a,f] <= sum(y[a,f,f2] for f2 in deltaplus_flightarcs[a][f])) # flight f is chosen only if it is traversed


### Constraint #3: maximum limit on the number of arrivals and departures from the airports

Finally, we add the airport capacity constraints.
For each airport, we know the total number of arrivals and departures on a regular day.
However, on the disruption day, only a fraction of flights can land and take-off, given by the parameters $\alpha$.
For example, if $\alpha = 0.5$, only half the flights can land or take-off.
This condition can be mathematically expressed using the following inequalities for every airport $i$,

\begin{aligned}
\sum_{\textrm{aircraft a}} \ \sum_{\textrm{flight }f \textrm{ that arrives at $i$}} x_{a,f} &\leq C^{arr}_{i} * \alpha \quad  \forall \ \textrm{airport } i, \\
\sum_{\textrm{aircraft a}} \ \sum_{\textrm{flight }f \textrm{ that departs from $i$}} x_{a,f} &\leq C^{dep}_{i} *\alpha \quad  \forall \ \textrm{airport } i.
\end{aligned}

The left hand side of the inequalities counts the total number of flights that land or take-off at the airports, and the right hand side sets the maximum limits.
As extreme cases, setting $\alpha  = 0$ implies that there is a complete shut-down of the airports, and $\alpha = 1$ implies that there is no disruption.

We can add these constraints to the model, with a default values set to $0.5$. Later in the notebook, we see how the disruption parameter affects the optimal flight plan.


In [17]:
alpha = .5

for i in N:
    total_departures = len([f for a in aircrafts for f in aircraft_flights[a] if flight_origin[f] == i])
    total_arrivals = len([f for a in aircrafts for f in aircraft_flights[a] if flight_dest[f] == i])

    model.addConstr(sum(x[a,f] for a in aircrafts for f in aircraft_flights[a] if flight_origin[f] == i) <= alpha*total_departures)
    model.addConstr(sum(x[a,f] for a in aircrafts for f in aircraft_flights[a] if flight_dest[f] == i) <= alpha*total_arrivals)


### Fire up Gurobi engines

We have added the decision variables, objective function, and the constraints to the model.
The model is ready to be solved.

In [18]:
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 536 rows, 1930 columns and 5772 nonzeros
Model fingerprint: 0x8a248b3c
Variable types: 0 continuous, 1930 integer (1930 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+03, 6e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Found heuristic solution: objective 1878476.0000
Presolve removed 498 rows and 1569 columns
Presolve time: 0.03s
Presolved: 38 rows, 361 columns, 722 nonzeros
Found heuristic solution: objective 561087.00000
Variable types: 0 continuous, 361 integer (343 binary)

Explored 0 nodes (0 simplex iterations) in 0.06 seconds (0.01 work units)
Thread count was 2 (of 2 available processors)

Solution count 2: 561087 1.87848e+06 

Optimal solution found (tolera

### Optimal solution

The solver solved the optimization problem in less than a second.
Let us now analyze the optimal solution.

In [19]:
operated_flights = {a: [f for f in aircraft_flights[a] if x[a,f].X > .5 if str(f)[0] != 's'] for a in aircrafts}

print("\nNet revenue total loss: $",round(model.objVal/10**6,2),'million')
print("Optimal number of flights served:",sum(len(operated_flights[a]) for a in aircrafts))
print("Optimal number of passengers transported:",sum(sum(flight_n_pass[f] for f in aircraft_flights[a] if x[a,f].X > .5) for a in aircrafts))
print("Optimal number of aircrafts utilized:",sum([1 if len(operated_flights[a]) > 0 else 0 for a in aircrafts]))



Net revenue total loss: $ 0.56 million
Optimal number of flights served: 51
Optimal number of passengers transported: 6570.0
Optimal number of aircrafts utilized: 9


## Full model

While this notebook walked through how to build an optimization model piece-by-piece, the following code contains the overall optimization model. You can input different parameter values and see how the optimal solution changes.
The value of $\alpha$ (percentage disruption) can be controlled using the slider below the cell.


In [20]:
import gurobipy as gp
from gurobipy import GRB
from ipywidgets import interact, interactive, fixed, interact_manual, widgets

N = G.nodes()

def solve_flight_planning(x):
    alpha = x

    model = gp.Model("airline_disruption")
    x, y = {}, {}
    for a in aircrafts:
        for f in aircraft_flights[a]:
            x[a,f] = model.addVar(name="x_%s,%s"%(a,f), vtype=GRB.BINARY)

        for (f1,f2) in flight_arcs_for_each_aircraft[a]:
            y[a,f1,f2] = model.addVar(name="y_%s,%s,%s"%(a,f1,f2), vtype=GRB.BINARY)

    model.update()

    objective = gp.quicksum((1-x[a,f])*flight_revenue[f] for a in aircrafts for f in aircraft_flights[a] if f in flight_revenue) # operating cost
    model.setObjective(objective, sense=GRB.MINIMIZE)

    for a in aircrafts:
        model.addConstr(sum(y[a,'source_%s'%a,f2] for f2 in deltaplus_flightarcs[a]['source_%s'%a]) == 1)
        model.addConstr(sum(y[a,f1,'sink_%s'%a] for f1 in deltaminus_flightarcs[a]['sink_%s'%a]) == 1)
        for f in aircraft_flights[a]:
            if str(f)[0] != 's':
                model.addConstr(sum(y[a,f,f2] for f2 in deltaplus_flightarcs[a][f]) == sum(y[a,f1,f] for f1 in deltaminus_flightarcs[a][f]))

    for a in aircrafts:
        for f in aircraft_flights[a]:
            model.addConstr(x[a,f] <= sum(y[a,f,f2] for f2 in deltaplus_flightarcs[a][f])) # flight f is chosen only if it is traversed


    for i in N:
        total_departures = len([f for a in aircrafts for f in aircraft_flights[a] if flight_origin[f] == i])
        total_arrivals = len([f for a in aircrafts for f in aircraft_flights[a] if flight_dest[f] == i])

        model.addConstr(sum(x[a,f] for a in aircrafts for f in aircraft_flights[a] if flight_origin[f] == i) <= alpha*total_departures)
        model.addConstr(sum(x[a,f] for a in aircrafts for f in aircraft_flights[a] if flight_dest[f] == i) <= alpha*total_arrivals)

    model.setParam('OutputFlag', 0)
    model.optimize()

    operated_flights = {a: [f for f in aircraft_flights[a] if x[a,f].X > .5 if str(f)[0] != 's'] for a in aircrafts}
    actual_rev = sum(flight_revenue[f] for f in flight_revenue) # operating cost
    print("Loss ($): ",round(model.objVal/10**6,2),'million')
    print("Optimal number of flights served:",sum(len(operated_flights[a]) for a in aircrafts))
    print("Optimal number of passengers transported:",sum(sum(flight_n_pass[f] for f in aircraft_flights[a] if x[a,f].X > .5) for a in aircrafts))
    print("Optimal number of aircrafts utilized:",sum([1 if len(operated_flights[a]) > 0 else 0 for a in aircrafts]))

    # the following lines are for visualization and needs the package pygraphviz (can be commented in case of troubles)
    print("Full network of operated flights:")
    G = nx.MultiDiGraph()
    arcs=[(flight_origin[f],flight_dest[f]) for a in aircrafts for f in operated_flights[a]]

    aircraft_color = {aircrafts[i]:"#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(aircrafts))}
    for a in aircrafts:
        for f in operated_flights[a]:
            G.add_edge(flight_origin[f],flight_dest[f],color=aircraft_color[a])
    A_graph = to_agraph(G)
    A_graph.layout('dot')
    display(A_graph)

print("Select a value for the level of disruption at the airports:\n")
print("Select 0 for complete shutdown of all airports; select 1 for business-as-usual.\n")

interact(solve_flight_planning, x=(0,1,0.05))


Select a value for the level of disruption at the airports:

Select 0 for complete shutdown of all airports; select 1 for business-as-usual.



interactive(children=(FloatSlider(value=0.0, description='x', max=1.0, step=0.05), Output()), _dom_classes=('w…

<function __main__.solve_flight_planning(x)>

In [21]:
model.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment


## Extensions

- Besides cancelling flights, there are other ways to alter the schedule after a disruption, such as delaying a flight, rebooking the passengers onto other flights, etc. Can you model these using decision variables? What type of constraints are needed?
- We only consider the cost of cancelling a flight. There may be other costs due to maintainance and repair, that we don't consider. How can other costs be incorporated into the model?
- There are indirect considerations to crew planning that this model does not include. For example, when reserve crew misses a connection due to a flight cancellation, that crew won't be able to serve a future flight. Can crew planning be captured in the model as well?


Copyright © 2023 Gurobi Optimization, LLC